# 🧪 Notebook 6 — Model Deployment & Inference

**Objective:**  

Prepare the final, tuned model for deployment. This notebook will:
 
- Load the best model(s) from Notebook 5.  
  
- Build preprocessing + prediction pipeline.  
  
- Create functions for inference on new data.  
  
- Test predictions with sample inputs.  
  
- Visualize feature importance for interpretability.  
  
- Save the pipeline for deployment.
 
 
This ensures that new data can be fed into the model consistently and predictions are reproducible.

---


## 6.1 Load Tuned Model & Scaler

Loading the pre-fitted scaler ensures any new data uses the same normalization applied during training. This guarantees consistency and avoids data leakage.


In [5]:
import joblib
import pandas as pd

# Load scaler
scaler = joblib.load("models/scaler.pkl")

# Load best models
best_log = joblib.load("models/tuned/best_log_reg.pkl")
best_rf = joblib.load("models/tuned/best_rf.pkl")

print("✅ Best models and scaler loaded successfully")

✅ Best models and scaler loaded successfully


---

## 6.2 Build Inference Pipeline

This pipeline ensures raw user input can be processed consistently and fed directly into the model.

In [6]:
def preprocess_input(new_data, scaler):
    """
    Accepts new raw input data (as DataFrame) and returns scaled features.
    """
    # Ensure order of features matches training set
    X_new = new_data.copy()
    
    # Scaling
    X_scaled = scaler.transform(X_new)
    return X_scaled

def predict_heart_disease(model, X_scaled):
    """
    Returns predicted class and probability for heart disease.
    """
    pred_class = model.predict(X_scaled)
    pred_proba = model.predict_proba(X_scaled)[:,1]
    return pred_class, pred_proba